Data needed in input:
- Segmentation masks from epifluorescence timelapses obtained by using the FIJI macro "Fiji utils/RNA_epifluorescence_timelapse_segmentation.ijm") within `experiment_dir`. 

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import porespy as ps
import scipy.ndimage as spim

from skimage.measure import regionprops

In [ ]:
# from porespy - seems like it's not a directly accessible function, but directly returns the chord lengths
def chord_counts(im):
    labels, N = spim.label(im > 0)
    props = regionprops(labels)
    chord_lens = np.array([i.filled_area for i in props])
    return chord_lens

In [ ]:
# CHANGE TO ABSOLUTE PATH containing binary masks from timelapse segmentation
experiment_dir = "/ABSOLUTE/PATH/TO/BINARY/MASKS/"

# Example: Bulk Assembly of condensates from RNA nanostars A, B, C

In [ ]:
os.chdir(experiment_dir)
files = list(os.listdir())
files_a = [x for x in files if 'NS_A' in x]
files_b = [x for x in files if 'NS_B' in x]
files_c = [x for x in files if 'NS_C' in x]
files_a.sort()
files_b.sort()
files_c.sort()

In [ ]:
cld = {}
for sample_files, sample_label in zip([files_a, files_b, files_c], ['NS_A', 'NS_B', 'NS_C']): 
    cld[sample_label] = {}
    print(sample_label)
    for filename, rep in zip(sample_files, range(1, len(sample_files)+1)): 
        print(filename, rep)
        # Initialise dictionary entry for this sample and this replicate FOV (`rep`)
        cld[sample_label][rep] = {'x':[], 'y':[], 'count_x': [], 'count_y' : []}
        # Read file and load timelapse of binary masks
        timelapse = cv2.imreadmulti(filename)[1]
        for timepoint in range(len(timelapse)): 
            # Invert mask -  from dark objects on bright background 
            # to bright objects on dark background
            img = np.array(255 - timelapse[timepoint][:, :, 0])
            # Extract X chords
            crds_x = ps.filters.apply_chords(im=img, spacing=1, axis=0, trim_edges=False)
            # Extract Y chords
            crds_y = ps.filters.apply_chords(im=img, spacing=1, axis=1, trim_edges=False)
            # Get CLD along X and save it in the entry
            data_x = ps.metrics.chord_length_distribution(crds_x, bins=100)
            cld[sample_label][rep]['x'].append(data_x)
            # Get CLD along Y and save it in the entry
            data_y = ps.metrics.chord_length_distribution(crds_y, bins=100)
            cld[sample_label][rep]['y'].append(data_y)
            # Get raw chord counts and save them in the corresponding entries
            cld[sample_label][rep]['count_x'].append(chord_counts(crds_x))
            cld[sample_label][rep]['count_y'].append(chord_counts(crds_y))
            # plot results every 15 timepoints
            if timepoint % 15 == 0:
                print('Timepoint: ', timepoint, "/", len(timelapse))
                plt.subplots(1, 4, figsize = (20, 5))
                plt.subplot(141)
                plt.imshow(img)
                plt.title('Original (inverted) mask')
                plt.subplot(142)
                plt.imshow(crds_x, interpolation='none', origin='lower')
                plt.title('Chords - X')
                plt.axis(False)
                plt.subplot(143)
                plt.imshow(crds_y, interpolation='none', origin='lower')
                plt.title('Chords - Y')
                plt.axis(False);
                plt.subplot(144)
                plt.bar(x=data_y.L, height=data_y.cdf, width=data_y.bin_widths, color='b', edgecolor='k', alpha=0.5, label = 'Y');
                plt.bar(x=data_x.L, height=data_x.cdf, width=data_x.bin_widths, color='r', edgecolor='k', alpha=0.5, label = 'X');
                plt.xlabel("Chord Length Distribution")
                plt.ylabel("Frequency");
                plt.legend(frameon = False)
                plt.show()

In [ ]:
cld.keys()

In [ ]:
# Save results in NPY format for processing and plotting
np.save('cld_bulk_sticky_nanostars.npy', cld, allow_pickle = True)

In [ ]:
# Load saved results to check they were correctly saved
cld_bulk_single = np.load('cld_bulk_sticky_nanostars.npy', allow_pickle = True).item()
cld_bulk_single['NS_A'][1].keys()